# Evaluate OCR text Detector quick example

In [1]:
import os
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
import sys
import glob
import torch
import warnings
from datetime import datetime
from matplotlib import pyplot as plt
from torch import nn
from torchvision.models import shufflenet_v2_x2_0

warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../../')
sys.path.append(NOMEROFF_NET_DIR)

from nomeroff_net.data_loaders import TextImageGenerator
from nomeroff_net.pipes.number_plate_text_readers.base.ocr import OCR

In [5]:
plt.rcParams["figure.figsize"] = (10, 10)

In [6]:
%matplotlib inline 

In [11]:
# auto download latest dataset
from nomeroff_net.tools import modelhub

# auto download latest dataset
info = modelhub.download_dataset_for_model("kz")
PATH_TO_DATASET = info["dataset_path"]

model_info = modelhub.download_model_by_name("kz")
MODEL_PATH = model_info["path"]

In [12]:
kz = OCR(model_name="kz", letters=model_info["letters"],
         linear_size=model_info["linear_size"], max_text_len=model_info["max_text_len"],
         height=model_info["height"], width=model_info["width"],
         color_channels=model_info["color_channels"],
         hidden_size=model_info["hidden_size"], backbone=model_info["backbone"])
kz.prepare(PATH_TO_DATASET, use_aug=False, num_workers=1)
kz.load()

GET ALPHABET
Max plate length in "val": 8
Max plate length in "train": 8
Max plate length in "test": 8
Letters train  {'Z', '2', '5', 'O', 'W', '9', 'C', 'L', 'S', '7', 'M', '8', 'R', '6', 'F', 'Y', 'Q', 'H', '4', 'X', 'U', 'J', '3', '0', 'K', 'P', 'T', '1', 'A', 'G', 'N', 'V', 'E', 'B', 'I', 'D'}
Letters val  {'Z', '2', '5', 'O', 'W', '9', 'C', 'L', 'S', '7', 'M', '8', '6', 'R', 'Y', 'F', 'Q', 'H', '4', 'X', 'U', 'J', '3', '0', 'K', 'P', 'T', '1', 'A', 'G', 'N', 'V', 'E', 'B', 'I', 'D'}
Letters test  {'Z', '2', '5', 'O', 'W', '9', 'C', 'S', '7', 'M', '8', '6', 'R', 'F', 'Q', 'Y', 'H', '4', 'X', 'U', 'J', '3', '0', 'K', 'P', 'T', '1', 'A', 'G', 'N', 'V', 'E', 'D', 'B', 'I', 'L'}
Max plate length in train, test and val do match
Letters in train, val and test do match
Letters: 0 1 2 3 4 5 6 7 8 9 A B C D E F G H I J K L M N O P Q R S T U V W X Y Z
START BUILD DATA


100%|██████████| 279/279 [00:00<00:00, 18860.37it/s]


DATA PREPARED


NPOcrNet(
  (conv_nn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [9]:
kz.test_acc(verbose=False)

Testing Accuracy:  0.967741935483871 in 279


0.967741935483871

In [10]:
for dataset_by_np_format in glob.glob(os.path.join(info["dataset_path"], "test_by_np_format/*")):
    print(os.path.basename(dataset_by_np_format))
    image_generator = TextImageGenerator(
            dataset_by_np_format,
            kz.letters,
            kz.max_text_len,
            img_w=kz.width,
            img_h=kz.height,
            batch_size=kz.batch_size,
            with_aug=False)
    acc = kz.acc_calc(image_generator)
    print('Validaton Accuracy: ', acc, "in", len(image_generator))

###@@@##


100%|██████████| 279/279 [00:00<00:00, 694.76it/s]


Validaton Accuracy:  0.9641434262948207 in 251
@###@@


100%|██████████| 279/279 [00:00<00:00, 63234.13it/s]


Validaton Accuracy:  1.0 in 1
###@@##


...

Validaton Accuracy:  1.0 in 12
@######


100%|██████████| 279/279 [00:00<00:00, 56211.49it/s]


Validaton Accuracy:  1.0 in 1
@###@@@


100%|██████████| 279/279 [00:00<00:00, 9853.25it/s]


Validaton Accuracy:  1.0 in 14
